In [ ]:
from urllib.parse import urlparse, urlunparse, urlencode
from urllib.request import urlretrieve
from pathlib import Path
import pandas as pd

import matplotlib.pyplot as plt
import re

In [ ]:
# Connet to Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

In [ ]:


from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)

In [ ]:
!pip install datefinder
!pip install googletrans==3.1.0a0

import datefinder
import re
from googletrans import Translator, constants
from pprint import pprint

In [ ]:
base_url = urlparse("https://www.numisbids.com/n.php")

def get_lot_url(auction_id, lot):
    params = {"p":"lot", "sid": auction_id, "lot" : lot }
    url_new_query = urlencode(params)
    url_parse = base_url._replace(query=url_new_query)
    new_url = urlunparse(url_parse)
    return new_url

def get_search_url(auction_id, lot):
    params = {"p":"lot", "sid": auction_id, "lot" : lot }
    url_new_query = urlencode(params)
    url_parse = base_url._replace(query=url_new_query)
    new_url = urlunparse(url_parse)
    return new_url

In [ ]:
auction_id = 4336
search_key = 'Elisabeth'
country = "Russia"
awers_regex = r"Awers:[^;]*"
rewers_regex = r"Rewers:[^;]*"
popiersie_key = ["popiersie","głowa","Popiersie","głowa"]
def_year = "nan"
grade_regex = r"\bAU\b|\bFA\b|\bPR\b|\bAG\b|\bG\b|\bVG\b|\bF\b|\bVF\b|\bXF\b|\bAU\b|\bMS\b"
search_url = urlparse("https://www.coinarchives.com/w/results.php?search=bust+AND+Russia+AND+Peter+II&s=1&upcoming=0&results=100")
#search_url = urlparse("https://www.numisbids.com/n.php" + "?search=%27" + search_key + "%27&p=sale&sid=" + str(auction_id) + "&s=b")
print(urlunparse(search_url))
driver.get(urlunparse(search_url))
#price = driver.find_element_by_xpath("//div[@class='viewlottext']/div[1]/div[@class='right']").text.split('\n')

lot_elements = driver.find_elements_by_class_name('auctiontitle')
#price = driver.find_element_by_xpath("//div[@class='viewlottext']/div[1]/div[@class='right']").text.split('\n')
#'https://www.numisbids.com/n.php?search=%27Catherine%27&p=sale&sid=4607&s=b'
lot_list = []
for e in lot_elements:
  lot_list.append(e.text.split(' ')[-1])


print(lot_list)
#lot_list = ['1128', '1129', '1130', '1131', '1132', '1133', '1134', '1135', '1136']

print(lot_list)

#img = driver.find_element_by_xpath("//img[starts-with(@src,'sales/hosted')]")
#price = driver.find_element_by_xpath("//div[@class='viewlottext']/div[1]/div[@class='right']").text.split('\n')
driver.find
#start_price = price[0].split(": ")[1]
#realized_price = price[1].split(": ")[1] if ":" in price[1] else "NaN"
#description = driver.find_element_by_xpath("//div[@class='description'][2]").text.replace("\n"," ")

In [ ]:
translator = Translator()
#print(f"{translation.origin} ({translation.src}) --> {translation.text} ({translation.dest})")

Path(str(auction_id)).mkdir(parents=True, exist_ok=True)
filename = "{}/dataset.csv".format(auction_id)
filename = "/content/gdrive/MyDrive/magisterka/dataset.csv"
with open(filename, "a") as out_file:
    try:
        last_uploaded = len(pd.read_csv(filename, delimiter='|', header=None))
    except:
        last_uploaded = 0
    if last_uploaded == 0:
        last_uploaded = 1
    print(last_uploaded)
    for lot in lot_list:
        driver.get(get_lot_url(auction_id, lot))
        img = driver.find_element_by_xpath("//img[starts-with(@src,'sales/hosted')]")
        price = driver.find_element_by_xpath("//div[@class='viewlottext']/div[1]/div[@class='right']").text.split('\n')
        start_price = price[0].split(": ")[1]
        realized_price = price[1].split(": ")[1] if ":" in price[1] else "NaN"
        description = driver.find_element_by_xpath("//div[@class='description'][2]").text.replace("\n"," ")
        descriptioneng = translator.translate(description).text
      #  if (not any(x in description for x in popiersie_key)):
      #    continue
        
        r1 = re.findall(r"1\d{3}",description)
        if (len(r1) > 0):
          year = str(r1[0])
        else:
          year = str(def_year)

        if False:
          r2 = re.findall(awers_regex,description)
          awers_str = str(r2).split(' ', 1)[1]
          r3 = re.findall(rewers_regex,description)
          rewers_str = str(r3).split(' ', 1)[1]
          if (not any(x in awers_str for x in popiersie_key)) and any(x in rewers_str for x in popiersie_key):
            temp = awers_str
            awers_str = rewers_str
            rewers_str = temp
        
        grade_str = description
        #grade_str = grade_str.replace(awers_str, '')
        #grade_str = grade_str.replace(rewers_str,'')
        r4 = re.findall(grade_regex,grade_str)
        if (len(r4) > 0):
          grade = str(r4[-1])
        else:
          grade = "nan"

        #print("awers " + awers_str)
        #print("rewers " + rewers_str)
        print("grade " + grade)
        print("rok " + year)

        
        awers_str = "no data"
        rewers_str = "no data"
        src = img.get_attribute('src')
        image_name = str(auction_id)+"__"+src.split("/")[-1]
        #urlretrieve(src,  "{}/{}".format(auction_id, image_name))
        row = [str(auction_id)+"__"+str(lot), search_key,  country, grade, year, image_name, src, start_price, realized_price, awers_str, rewers_str,descriptioneng, description]

        #print("|".join(row)+"\n")
        out_file.write("|".join(row)+"\n")

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

df = pd.read_csv('/content/gdrive/MyDrive/magisterka/clear_dataset.csv', header=None, delimiter = "|")
#df.sheet_names
print(df.groupby(1)[0].nunique())
print(df.groupby(2)[0].nunique())
print(df.groupby(2)[1].nunique())
df.head(500)

In [ ]:
}from matplotlib import pyplot as plt
import cv2
from skimage import io

img = io.imread(src)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
print((img.shape[1]/2))
crop_img = img[:, :int(img.shape[1]/2)]
plt.imshow(crop_img)
plt.axis('off')
plt.show()

In [ ]:
driver.quit()

In [ ]:
df[1].unique()
#df[2].unique()

In [ ]:
with open(filename, "r") as f:
    lines = f.readlines()
with open(filename, "w") as f:
    for i,line in enumerate(lines):
      if i>69:
        f.write(line)

In [ ]:
for index, row in df.iterrows():
  Path("/content/gdrive/MyDrive/magisterka/images/{}".format(row[1])).mkdir(parents=True, exist_ok=True)
  urlretrieve(row[6],  "/content/gdrive/MyDrive/magisterka/images/{}/{}".format(row[1], row[5]))

In [ ]:
import os